# This notebook generates the data for 1D Poisson's equation

In [ ]:
!pip install pathos --quiet

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from pathos.pools import ProcessPool
from scipy import linalg, interpolate
from sklearn import gaussian_process as gp
import matplotlib.pyplot as plt
import numpy as np
from scipy import interpolate
import sys
import time
from functools import wraps
from scipy.interpolate import UnivariateSpline


In [ ]:
def timing(f):
    """Decorator for measuring the execution time of methods."""

    @wraps(f)
    def wrapper(*args, **kwargs):
        ts = time.time()
        result = f(*args, **kwargs)
        te = time.time()
        print("%r took %f s\n" % (f.__name__, te - ts))
        sys.stdout.flush()
        return result

    return wrapper

In [ ]:
class GRF(object):
    def __init__(self, T, kernel="RBF", length_scale=1, N=1000, interp="cubic"):
        self.N = N
        self.interp = interp
        self.x = np.linspace(0, T, num=N)[:, None]
        if kernel == "RBF":
            K = gp.kernels.RBF(length_scale=length_scale)
        elif kernel == "AE":
            K = gp.kernels.Matern(length_scale=length_scale, nu=0.5)
        self.K = K(self.x)
        self.L = np.linalg.cholesky(self.K + 1e-13 * np.eye(self.N))

    def random(self, n):
        """Generate `n` random feature vectors.
        """
        u = np.random.randn(self.N, n)
        return np.dot(self.L, u).T

    def eval_u_one(self, y, x):
        """Compute the function value at `x` for the feature `y`.
        """
        if self.interp == "linear":
            return np.interp(x, np.ravel(self.x), y)
        f = interpolate.interp1d(
            np.ravel(self.x), y, kind=self.interp, copy=False, assume_sorted=True
        )
        return f(x)

    def eval_u(self, ys, sensors):
        """For a list of functions represented by `ys`,
        compute a list of a list of function values at a list `sensors`.
        """
        if self.interp == "linear":
            return np.vstack([np.interp(sensors, np.ravel(self.x), y).T for y in ys])
        p = ProcessPool(nodes=4)
        res = p.map(
            lambda y: interpolate.interp1d(
                np.ravel(self.x), y, kind=self.interp, copy=False, assume_sorted=True
            )(sensors).T,
            ys,
        )
        return np.vstack(list(res))


def space_samples(space, T):
    features = space.random(100000)
    sensors = np.linspace(0, T, num=1000)
    u = space.eval_u(features, sensors[:, None])

    plt.plot(sensors, np.mean(u, axis=0), "k")
    plt.plot(sensors, np.std(u, axis=0), "k--")
    plt.plot(sensors, np.cov(u.T)[0], "k--")
    plt.plot(sensors, np.exp(-0.5 * sensors ** 2 / 0.2 ** 2))
    for ui in u[:3]:
        plt.plot(sensors, ui)
    plt.show()


In [ ]:
def solver(f, N):
    """u_xx = 20f, x \in [0, 1]
    u(0) = u(1) = 0
    """
    h = 1 / (N - 1)
    K = -2 * np.eye(N - 2) + np.eye(N - 2, k=1) + np.eye(N - 2, k=-1)
    b = h ** 2 * 20 * f[1:-1]
    u = np.linalg.solve(K, b)
    u = np.concatenate(([0], u, [0]))
    return u


def example():
    space = GRF(1, length_scale=0.05, N=1000, interp="cubic")
    m = 100

    features = space.random(1)
    sensors = np.linspace(0, 1, num=m)
    sensor_values = space.eval_u(features, sensors[:, None])
    y = solver(sensor_values[0], m)
    np.savetxt("poisson_high.dat", np.vstack((sensors, np.ravel(sensor_values), y)).T)

    m_low = 10
    x_low = np.linspace(0, 1, num=m_low)
    f_low = space.eval_u(features, x_low[:, None])
    y_low = solver(f_low[0], m_low)
    np.savetxt("poisson_low.dat", np.vstack((x_low, y_low)).T)


@timing
def gen_data():
    print("Generating operator data...", flush=True)
    space = GRF(1, length_scale=0.05, N=1000, interp="cubic")
    m = 100
    num = 100

    features = space.random(num)
    sensors = np.linspace(0, 1, num=m)
    sensor_values = space.eval_u(features, sensors[:, None])

    x = []
    y = []
    for i in range(num):
        tmp = solver(sensor_values[i], m)
        idx = np.random.randint(0, m, size=1)
        x.append(sensors[idx])
        y.append(tmp[idx])
    x = np.array(x)
    y = np.array(y)

    m_low = 10
    x_low = np.linspace(0, 1, num=m_low)
    f_low = space.eval_u(features, x_low[:, None])
    y_low = []
    y_low_x = []
    for i in range(num):
        tmp = solver(f_low[i], m_low)
        tmp = interpolate.interp1d(x_low, tmp, copy=False, assume_sorted=True)
        y_low.append(tmp(sensors))
        y_low_x.append(tmp(x[i]))
    y_low = np.array(y_low)
    y_low_x = np.array(y_low_x)
    np.savez_compressed(
        "/content/train.npz", X0=sensor_values, X1=x, y=y, y_low=y_low, y_low_x=y_low_x
    )

    for i in range(5):
         plt.figure()
         plt.plot(sensors, sensor_values[i], "k")
         plt.plot(x[i], y[i], "or")
         plt.plot(sensors, y_low[i], "b")
         plt.plot(x[i], y_low_x[i], "xb")
         plt.show()


def main():
    gen_data()
    

In [ ]:
def gen_data_fno(points_hi,points_lo,num,lscale):
    space = GRF(1, length_scale=lscale, N=1000, interp="cubic")
    m = points_hi
    n = num
    m_low = points_lo

    y_high = np.zeros((n,m))
    y_low_xlow = np.zeros((n,m_low))
    features = space.random(n)
    sensors = np.linspace(0, 1, num=m)
    sensor_values = space.eval_u(features, sensors[:, None])

    x_low = np.linspace(0, 1, num=m_low)
    f_low = space.eval_u(features, x_low[:, None])

    for i in range(n):
        t1 = time.time()
        y_high[i] = solver(sensor_values[i], m)
        t2 = time.time()
        t3 = time.time()
        y_low_xlow[i] = solver(f_low[i], m_low)
        t4 = time.time()
        if i % 100 == 0:
            print('Sample-{}, Time-HF-{:0.4f}, Time-LF-{:0.4f}'.format(i, t2-t1, t4-t3))

    np.savez_compressed(
        "data/train.npz", f_stoch =sensor_values, xhi=sensors, yhi=y_high, x_low=x_low, y_low=y_low_xlow
    )

In [ ]:
gen_data_fno(50,8,10000,0.1)

In [ ]:
with np.load('data/train.npz','r') as data:
  y_low = data['y_low']
  x_low = data['x_low']
  xhi = data['xhi']
  yhi = data['yhi']
  f =  data['f_stoch']

In [ ]:
ylo = np.zeros((10000,50))
for i in range(len(y_low)):
  spl = interpolate.interp1d(x_low, y_low[i],kind="cubic")
  ylo[i] = spl(xhi)

In [ ]:
n = 440
fig1 = plt.figure(figsize=(10,6), dpi=100)
plt.plot(x_low,y_low[n])
plt.plot(xhi,ylo[n])
plt.plot(xhi,yhi[n])
plt.plot(x_low,y_low[110])
plt.legend(['low_hi','hi','lo'])


In [ ]:
def mseloss(y_true,y_pred):
    return np.mean(np.square(y_pred-y_true))

In [ ]:
mseloss(ylo,yhi)

In [ ]:
np.savez_compressed(
        "rbf_possion_8pt_50pt__lscale_01_10000.npz", f_stoch =f, xhi=xhi, yhi=yhi, x_low=x_low, y_low_100=ylo, y_low_x = y_low
    )